In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import re
import warnings


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
warnings.filterwarnings("ignore", category=DeprecationWarning)

C:\Users\pengu\Anaconda3\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
C:\Users\pengu\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.o

In [3]:
data = pd.read_csv("./datasets/data_200.csv")


In [4]:
data.shape

(200, 3)

In [5]:
data.head()

,text_parsed,money_laundering,text_parsed_length
0,running mayor 2016 thennorfolk sheriff bob mcc...,1,2796
1,ed probing former legislator money laundering ...,1,2513
2,enforcement directorate issued lookout notice ...,1,1891
3,minutes landing private jet biggin hill airfie...,1,4585
4,lawyers defending founders classified site bac...,1,2580


### Split the data into train and test sets

In [6]:
X_train, X_test, y_train, y_test = train_test_split(data['text_parsed'], 
                                                    data['money_laundering'], 
                                                    test_size=0.25, 
                                                    random_state=42)

In [7]:
ngram_range = (1,2)
min_df = 0
max_df = 1.
max_features = 300

#### Text must be vectorized

In [8]:
tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)
                        
features_train = tfidf.fit_transform(X_train).toarray()
labels_train = y_train
print(features_train)

features_test = tfidf.transform(X_test).toarray()
labels_test = y_test
print(features_test.shape)


[[0.         0.09434589 0.         ... 0.06146513 0.06787635 0.        ]
 [0.08409251 0.         0.         ... 0.11849634 0.06235373 0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.31070768 ... 0.17533924 0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
(50, 300)


## Building Models

### Random Forest

In [9]:
from sklearn.ensemble import RandomForestClassifier
model_RF  = RandomForestClassifier(random_state=42)
model_RF.fit(features_train, labels_train)
RF_predictions = model_RF.predict(features_test)
print('Accuracy: ', accuracy_score(labels_test, RF_predictions))
print(classification_report(labels_test, RF_predictions))
warnings.filterwarnings("ignore", category=DeprecationWarning)
model_RF.get_params()

Accuracy:  0.84
              precision    recall  f1-score   support

           0       0.88      0.93      0.90        40
           1       0.62      0.50      0.56        10

   micro avg       0.84      0.84      0.84        50
   macro avg       0.75      0.71      0.73        50
weighted avg       0.83      0.84      0.83        50



C:\Users\pengu\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


{'bootstrap': True,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 10,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}

In [10]:
from sklearn.model_selection import GridSearchCV
n_estimators = [100, 300, 500]
max_depth = [5, 8, 15, 20]
min_samples_split = [2, 5, 10, 15, 50]
min_samples_leaf = [1, 2, 5, 10] 

RF_params = dict(n_estimators = n_estimators, max_depth = max_depth,  
              min_samples_split = min_samples_split, 
             min_samples_leaf = min_samples_leaf)

grid_RF = GridSearchCV(model_RF, RF_params, cv = 5, verbose = 1, 
                      n_jobs = -1)
best_RF = grid_RF.fit(features_train, labels_train)
warnings.filterwarnings("ignore", category=DeprecationWarning)

Fitting 5 folds for each of 240 candidates, totalling 1200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:    7.7s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:   16.0s
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:   27.7s
[Parallel(n_jobs=-1)]: Done 1200 out of 1200 | elapsed:   41.8s finished


In [11]:
best_RF.best_params_

{'max_depth': 5,
 'min_samples_leaf': 2,
 'min_samples_split': 50,
 'n_estimators': 300}

#### Final Random Forest Model

In [12]:
from sklearn.ensemble import RandomForestClassifier
warnings.filterwarnings("ignore", category=DeprecationWarning)
final_RF  = RandomForestClassifier(random_state=42,
                                 max_depth= 5,
                                 min_samples_leaf= 2,
                                 min_samples_split= 50,
                                 n_estimators= 300)

final_RF.fit(features_train, labels_train)
final_RF_predictions = final_RF.predict(features_test)
print('Accuracy: ', accuracy_score(labels_test, final_RF_predictions))
print(classification_report(labels_test, final_RF_predictions))

Accuracy:  0.88
              precision    recall  f1-score   support

           0       0.93      0.93      0.93        40
           1       0.70      0.70      0.70        10

   micro avg       0.88      0.88      0.88        50
   macro avg       0.81      0.81      0.81        50
weighted avg       0.88      0.88      0.88        50



### Logistic Regression Model

In [13]:
model_LR = LogisticRegression(solver='liblinear')
model_LR.fit(features_train, labels_train)
LR_predictions = model_LR.predict(features_test)
print('Accuracy: ', accuracy_score(labels_test, LR_predictions))
print(classification_report(labels_test, LR_predictions))
model_LR.get_params()

Accuracy:  0.84
              precision    recall  f1-score   support

           0       0.88      0.93      0.90        40
           1       0.62      0.50      0.56        10

   micro avg       0.84      0.84      0.84        50
   macro avg       0.75      0.71      0.73        50
weighted avg       0.83      0.84      0.83        50



{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'max_iter': 100,
 'multi_class': 'warn',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'liblinear',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

#### Logistic Regression Tuning

In [16]:
LR_params = {'C': [0.1,0.001,1],
              'penalty' : ['l1', 'l2'],
              'solver': ['liblinear']}
grid_LR = GridSearchCV(model_LR, LR_params, cv = 5, verbose = 1)

best_LR = grid_LR.fit(features_train, labels_train)
best_LR.best_params_

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.0s finished


{'C': 1, 'penalty': 'l1', 'solver': 'liblinear'}

#### Final Logistic Regression Model

In [17]:
final_LR = LogisticRegression(C=1, penalty='l1', solver='liblinear')
final_LR.fit(features_train, labels_train)
final_LR_predictions = final_LR.predict(features_test)
print('Accuracy: ', accuracy_score(labels_test, final_LR_predictions))
print(classification_report(labels_test, final_LR_predictions))

Accuracy:  0.84
              precision    recall  f1-score   support

           0       0.88      0.93      0.90        40
           1       0.62      0.50      0.56        10

   micro avg       0.84      0.84      0.84        50
   macro avg       0.75      0.71      0.73        50
weighted avg       0.83      0.84      0.83        50



### KNeighborsClassifier Model

In [18]:
model_KNN = KNeighborsClassifier()
model_KNN.fit(features_train, labels_train)
KNN_predictions = model_KNN.predict(features_test)
print('Accuracy: ', accuracy_score(labels_test, KNN_predictions))
print(classification_report(labels_test, KNN_predictions))

Accuracy:  0.88
              precision    recall  f1-score   support

           0       0.97      0.88      0.92        40
           1       0.64      0.90      0.75        10

   micro avg       0.88      0.88      0.88        50
   macro avg       0.81      0.89      0.84        50
weighted avg       0.91      0.88      0.89        50



In [19]:
model_KNN.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

#### KNeighborsClassifier Tuning

In [20]:
from sklearn.model_selection import GridSearchCV
params_KNN = {'n_neighbors': [1, 2, 3, 4, 5, 6, 7], 
              'p': [1, 2, 5]}


grid_KNN = GridSearchCV(model_KNN, params_KNN, cv = 5, verbose = 1, 
                      n_jobs = -1)
best_KNN = grid_KNN.fit(features_train, labels_train)

Fitting 5 folds for each of 21 candidates, totalling 105 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done 105 out of 105 | elapsed:    0.6s finished


In [21]:
best_KNN.best_params_

{'n_neighbors': 3, 'p': 1}

#### Final KNeighbors Model

In [22]:
final_model_KNN = KNeighborsClassifier(n_neighbors=3,p=1)
final_model_KNN.fit(features_train, labels_train)
KNN_predictions = final_model_KNN.predict(features_test)
print('Accuracy: ', accuracy_score(labels_test, KNN_predictions))
print(classification_report(labels_test, KNN_predictions))

Accuracy:  0.86
              precision    recall  f1-score   support

           0       0.90      0.93      0.91        40
           1       0.67      0.60      0.63        10

   micro avg       0.86      0.86      0.86        50
   macro avg       0.78      0.76      0.77        50
weighted avg       0.86      0.86      0.86        50



### Decision Tree

In [23]:
model_DT = DecisionTreeClassifier(random_state=42)
model_DT.fit(features_train, labels_train)
DT_predictions = model_DT.predict(features_test)
print('Accuracy: ', accuracy_score(labels_test, DT_predictions))
print(classification_report(labels_test, DT_predictions))
model_DT.get_params()

Accuracy:  0.84
              precision    recall  f1-score   support

           0       0.88      0.93      0.90        40
           1       0.62      0.50      0.56        10

   micro avg       0.84      0.84      0.84        50
   macro avg       0.75      0.71      0.73        50
weighted avg       0.83      0.84      0.83        50



{'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'presort': False,
 'random_state': 42,
 'splitter': 'best'}

#### Decision Tree Tuning

In [25]:
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

min_samples_leaf = [1, 3, 8]
criterion = ['gini', 'entropy']
max_depth = [3,None]



DT_params = dict(max_depth=max_depth,
                      criterion=criterion,
                      min_samples_leaf=min_samples_leaf)

model_DT  = DecisionTreeClassifier(random_state=42)
grid_DT = RandomizedSearchCV(model_DT, DT_params, cv = 5, verbose = 1)
best_DT = grid_DT.fit(features_train, labels_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.0s finished


In [26]:
best_DT.best_params_

{'min_samples_leaf': 1, 'max_depth': 3, 'criterion': 'gini'}

#### Final Desicion Tree Model

In [27]:
final_model_DT = DecisionTreeClassifier(random_state=42,
                                        criterion= 'gini', 
                                        max_depth= 3,
                                        min_samples_leaf=1)
final_model_DT.fit(features_train, labels_train)
final_DT_predictions = final_model_DT.predict(features_test)
print('Accuracy: ', accuracy_score(labels_test, final_DT_predictions))
print(classification_report(labels_test, final_DT_predictions))

Accuracy:  0.84
              precision    recall  f1-score   support

           0       0.88      0.93      0.90        40
           1       0.62      0.50      0.56        10

   micro avg       0.84      0.84      0.84        50
   macro avg       0.75      0.71      0.73        50
weighted avg       0.83      0.84      0.83        50



### GaussianNB

In [28]:
model_GNB = GaussianNB()
model_GNB.fit(features_train, labels_train)
model_GNB_predictions = model_GNB.predict(features_test)
print('Accuracy: ', accuracy_score(labels_test, model_GNB_predictions))
print(classification_report(labels_test, model_GNB_predictions))

Accuracy:  0.84
              precision    recall  f1-score   support

           0       0.92      0.88      0.90        40
           1       0.58      0.70      0.64        10

   micro avg       0.84      0.84      0.84        50
   macro avg       0.75      0.79      0.77        50
weighted avg       0.85      0.84      0.85        50



#### Tuning GaussianNB model

In [30]:
from sklearn.model_selection import GridSearchCV
params_NB = {'var_smoothing': np.logspace(0,-9, num=100)}

grid_NB = GridSearchCV(model_GNB, params_NB, cv = 5, verbose = 1, 
                      n_jobs = -1)

bestNB = grid_NB.fit(features_train, labels_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    0.1s finished


In [31]:
bestNB.best_params_

{'var_smoothing': 0.01}

#### Final GNB model

In [32]:
final_model_GNB = GaussianNB(var_smoothing=0.01)
final_model_GNB.fit(features_train, labels_train)
final_GNB_predictions = final_model_GNB.predict(features_test)
print('Accuracy: ', accuracy_score(labels_test, final_GNB_predictions))
print(classification_report(labels_test, final_GNB_predictions))

Accuracy:  0.88
              precision    recall  f1-score   support

           0       0.95      0.90      0.92        40
           1       0.67      0.80      0.73        10

   micro avg       0.88      0.88      0.88        50
   macro avg       0.81      0.85      0.83        50
weighted avg       0.89      0.88      0.88        50

